In [110]:
import numpy as np
import pandas as pd
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

c:\Users\chati\AppData\Local\Programs\Python\Python310\lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


__Initialization__

In [254]:
# example from provided video
# seq1="ACACT" # y sequence
# seq2="AAT" # x sequence

# gap open (h) and extension (g) penalties 
# gap_open=-3
# gap_extend=-1


# extraneous case
# seq1="ACAATCT" # y sequence
# seq2="AAT" # x sequence

# from provided files
seq1="MAVHQLIRRP" # test_seq3.fa
seq2="MQLIRHP" # test_seq4.fa

# gap open (h) and extension (g) penalties 
gap_open=-10
gap_extend=-1


# initialize align matrix as mat of zeroes
align_matrix=np.zeros((len(seq2)+1, len(seq1)+1))
gapA_matrix=np.zeros((len(seq2)+1, len(seq1)+1))
gapB_matrix=np.zeros((len(seq2)+1, len(seq1)+1))

In [255]:
# initialize values

# first row and col of align matrix are -inf

# first row of gapA
for j in (range(len(seq1)+1)):
    align_matrix[0][j]=-np.inf

# first column of gapA
for i in (range(len(seq2)+1)):
    align_matrix[i][0]=-np.inf

# M(0,0)=0
align_matrix[0][0]=0

# first row of gapA
for j in (range(len(seq1)+1)):
    gapA_matrix[0][j]=-np.inf

# first column of gapA
for i in (range(len(seq2)+1)):
    gapA_matrix[i][0]=gap_open+gap_extend*i

# first column of gapB
for i in (range(len(seq2)+1)):
    gapB_matrix[i][0]=-np.inf

# first row of gapB
for j in (range(len(seq1)+1)):
    gapB_matrix[0][j]=gap_open+gap_extend*j 

__Alignmente Functions__

 - Functions were defined based on provided video [here](https://www.youtube.com/watch?v=NqYY0PJbD3s).

In [256]:
def score(b1, b2):
    s=-1
    if (b1==b2):
        s=1
    return s

def fill_M(align_matrix, gapA_matrix, gapB_matrix, i, j, seq1, seq2):
    M=align_matrix[i-1][j-1]+score(seq2[i-1], seq1[j-1])
    Ix=gapA_matrix[i-1][j-1]+score(seq2[i-1], seq1[j-1])
    Iy=gapB_matrix[i-1][j-1]+score(seq2[i-1], seq1[j-1])
    return max(M, Ix, Iy)

def fill_Ix(align_matrix, gapA_matrix, i, j, gap_open, gap_extend):
    M=align_matrix[i-1][j]+gap_open+gap_extend
    Ix=gapA_matrix[i-1][j]+gap_extend
    return max(M, Ix)

def fill_Iy(align_matrix, gapB_matrix, i, j, gap_open, gap_extend):
    M=align_matrix[i][j-1]+gap_open+gap_extend
    Iy=gapB_matrix[i][j-1]+gap_extend
    return max(M, Iy)

def fill_alignment_matrix(align_matrix, gapA_matrix, gapB_matrix, i, j, seq1, seq2, gap_open, gap_extend):
    Mij=fill_M(align_matrix, gapA_matrix, gapB_matrix, i, j, seq1, seq2)
    Ixij=fill_Ix(align_matrix, gapA_matrix, i, j, gap_open, gap_extend)
    Iyij=fill_Iy(align_matrix, gapB_matrix, i, j, gap_open, gap_extend)
    return Mij, Ixij, Iyij

In [257]:
# loop through indices of matrices
for i in range(1, len(seq2)+1):
    for j in range(1, len(seq1)+1):
        # print(i,j)
        align_matrix[i][j], gapA_matrix[i][j], gapB_matrix[i][j]=fill_alignment_matrix(align_matrix, gapA_matrix, gapB_matrix, i, j, seq1, seq2, gap_open, gap_extend)

In [277]:
# testing from implementation
from align import NeedlemanWunsch, read_fasta
nw=NeedlemanWunsch("./substitution_matrices/BLOSUM62.mat", gap_open, gap_extend)
nw.align(seq1, seq2)

(array([[  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf,   1., -12., -13., -14., -15., -16., -17., -18., -19., -20.],
        [-inf, -12.,   0., -11., -12., -11., -14., -15., -16., -17., -18.],
        [-inf, -13., -11.,  -1., -12., -13., -10., -15., -16., -17., -18.],
        [-inf, -14., -12., -12.,  -2., -13., -14.,  -9., -16., -17., -18.],
        [-inf, -15., -13., -13., -13.,  -3., -14., -15.,  -8., -15., -18.],
        [-inf, -16., -14., -14., -12., -14.,  -4., -15., -16.,  -9., -16.],
        [-inf, -17., -15., -15., -15., -13., -15.,  -5., -16., -17.,  -8.]]),
 array([[-10., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-11., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-12., -10., -23., -24., -25., -26., -27., -28., -29., -30., -31.],
        [-13., -11., -11., -22., -23., -22., -25., -26., -27., -28., -29.],
        [-14., -12., -12., -12., -23., -23., -21., -26., -27., -28., -29.],
        [-

In [278]:
# sub dict for alignment scoring
# nw.sub_dict

__Backtracing Functions__

 - Begin at the bottom right of the matrix and find the largest value among M, Ix, and Iy. 
 - If M is the largest value, we go left and up; if Ix is the largest we go up; and if Iy is the largest we go left.
 - Stop at top left of the matrix - (0,0).

In [287]:
# for the values across all three matrices at (i,j), get the max, and return an update vector that describes where to move
def get_next_move(align_matrix, gapA_matrix, gapB_matrix, i, j):
    mat_values={"M":align_matrix[i][j], "Ix":gapA_matrix[i][j], "Iy":gapB_matrix[i][j]}
    max_key=max(mat_values, key=mat_values.get)
    # check if there are multiple max values
    max_key_list=[key for key,val in mat_values.items() if val==mat_values[max_key]]
    return max_key_list

def query_sub_dict(match_val , mismatch_val, s1, s2, mtype="m", subdict={}): # mtypes: m, mm, sub - match, mismatch, substition
    if (mtype=="m"):
        return match_val
    if (mtype=="mm"):
        return mismatch_val
    if (mtype=="sub"):
        return subdict[(s1,s2)]

# WE MAY HAVE TO CHANGE SCORE FUNCTION ABOVE TO ALSO USE THE SUBSTITION VALUES FOR MATCH AND MISMATCH

def get_alignment_score(seq1_align, seq2_align, match_val, mismatch_val, gap_open, gap_extend, mtype="c", subdict={}):
    prev_label_seq1=None
    prev_label_seq2=None
    alignment_score=0
    alignment_print=[]
    for s1, s2 in zip(seq1_align, seq2_align):
        # if (mtype=="c"):
        if (s1!="-" and s2!="-"):
            if (s1==s2):
                # alignment_score+=match_val
                alignment_score+=subdict[(s1,s2)]
                alignment_print.append("|")
            else:
                # alignment_score+=mismatch_val
                alignment_score+=subdict[(s1,s2)]
                alignment_print.append("·")
        else:
            if (s1=="-"):
                if (prev_label_seq1!="gap"):
                    alignment_score+=gap_open+gap_extend
                    prev_label_seq1="gap"
                else:
                    alignment_score+=gap_extend
            if (s2=="-"):
                if (prev_label_seq2!="gap"):
                    alignment_score+=gap_open+gap_extend
                    prev_label_seq2="gap"
                else:
                    alignment_score+=gap_extend
            alignment_print.append(" ")
        # else:


    return alignment_score, alignment_print

In [289]:
def print_alignment(seq1_align, seq2_align, alignment_print):
    print(''.join(seq1_align))
    print(''.join(alignment_print))
    print(''.join(seq2_align))

In [290]:
# starting positions are the current positions (bottom right corner) - pos variable is changed while start_pos is our record of the starting position
start_pos=(len(seq2), len(seq1)) # -> curr_i=len(seq2), curr_j=len(seq1)
pos=start_pos
print("Starting position: ", start_pos)

# alignment lists for sequences (they should be matching in length so we can draw 1 to 1 alignments)
seq1_align=[]
seq2_align=[]
update_vectors={"M":(-1,-1), "Ix":(-1,0), "Iy":(0,-1)} # where M -> go up left, Ix -> go up, Iy -> left

# method="highroad" 
method="lowroad"

# backtrace using the highroad method
while (sum(pos)!=0):
    print(pos)
    max_key_list=get_next_move(align_matrix, gapA_matrix, gapB_matrix, pos[0], pos[1])

    # in the case of ties - for the highroad method, we prefer M; for the lowroad method, we prefer gaps
    if len(max_key_list)>1:
        if (method=="highroad"):
            max_key=max_key_list[0]
        if (method=="lowroad"):
            max_key=max_key_list[1]
    else:
        max_key=max_key_list[0]

    update_vector=update_vectors[max_key]

    pos=tuple(map(sum,zip(pos,update_vector)))
    # print(seq2[pos[0]], seq1[pos[1]])
    if (max_key=="M"):
        seq2_align.append(seq2[pos[0]])
        seq1_align.append(seq1[pos[1]])
    elif (max_key=="Ix"):
        seq2_align.append(seq2[pos[0]])
        seq1_align.append("-")
    elif (max_key=="Iy"):
        seq1_align.append(seq1[pos[1]])
        seq2_align.append("-")
        
print(pos)

# reverse lists
seq1_align.reverse()
seq2_align.reverse()


Starting position:  (7, 10)
(7, 10)
(6, 9)
(5, 8)
(4, 7)
(3, 6)
(2, 5)
(1, 4)
(1, 3)
(1, 2)
(1, 1)
(0, 0)


In [291]:
match_val=1
mismatch_val=-1
alignment_score, alignment_print=get_alignment_score(seq1_align, seq2_align, match_val, mismatch_val, gap_open, gap_extend, mtype="c", subdict=nw.sub_dict)
print(alignment_score)

17.0


In [270]:
print_alignment(seq1_align, seq2_align, alignment_print)

MAVHQLIRRP
|   ||||·|
M---QLIRHP


In [285]:
alignments=pairwise2.align.globalms(seq1, seq2, 1, -1, gap_open+gap_extend, gap_extend)
print(format_alignment(*alignments[0]))

MAVHQLIRRP
|   ||||.|
M---QLIRHP
  Score=-8



In [265]:
gap_extend
gap_open

-10

In [284]:
alignments=pairwise2.align.globalms("AAAAAAAAAAAAAAAAAAAAAAAACCGATCGATCGGATTCGATGG", "CCGATCGATCGGATTCGATGGTCGACTATCGTATTCTTAG", 1, -1, gap_open+gap_extend, gap_extend)
print(format_alignment(*alignments[3]))

AAAAAAAAAAAAAAAAAAAAAAAACCGATCGATCGGATTCGATGG
...|...|....|....|.......|.||||..     |...|.|
CCGATCGATCGGATTCGATGGTCGACTATCGTA-----TTCTTAG
  Score=-31

